In [1]:
#imports 
import json
import collections
import more_itertools
import seaborn
import pandas as pd
import random
import subprocess
import pathlib
import numpy as np
import datetime
import tqdm
import csv
import re
#from nltk.corpus import stopwords
#from nltk.stem import WordNetLemmatizer
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
import mmap
from tqdm import tqdm
import openpyxl
import more_itertools

In [2]:
# Function: get_num_lines
# Determines number of lines in a file to enable progress bars with tqdm

def get_num_lines(filename):
    fp = open(filename, "r+")
    buf = mmap.mmap(fp.fileno(),0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

In [3]:
#data -- import/upload xlsx spreadsheet
data = '/pscratch/sd/m/masare/sacct/incident_sla.xlsx'
#data = '/pscratch/sd/m/masare/sacct/incident_sla.csv'

In [4]:
CLEANED_TXT_CSV = 'data_text_clean.csv'
#SEARCH_TERM_LIST = 'software_exp_list.csv'
SEARCH_TERM_LIST = 'Brandon_Software_Terms.csv'
OUTPUT_FILE = 'AY24_Complete_BrandonTerms.csv'

In [5]:
random.seed(123)

In [6]:
incident = pd.read_excel(data)

In [7]:
# Function: get_num_lines
# Determines number of lines in a file to enable progress bars with tqdm

def get_num_lines(filename):
    fp = open(filename, "r+")
    buf = mmap.mmap(fp.fileno(),0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines


In [8]:
#clean 
   #isolate import columns, drop unimportant columns
   #remove time stamps (can look at my code and use that) 
text_cols = [   'Number',
                 'Created',
                'Title',
                'Additional comments',
                'Close notes',
                'Comments and Work notes',
                'Staff work notes (NERSC private)']

inc_text_df=incident[text_cols].copy()
inc_text_df.fillna('', inplace=True)
inc_text_df = inc_text_df.astype(str)



In [9]:
# :-----------------------------------------------------------------------------
#clean 
   #isolate import columns, drop unimportant columns
   #remove time stamps (can look at my code and use that) 
# Function: process_comment_text
# Input: ind -- index from the pandas dataframe that holds the cleaned xlsx data
# Output: Single row that includes, index, number, and text columns combined and
#         cleaned into a single column entry.

def process_comment_text(ind):
    inc_text_entry = str( \
                     inc_text_df.iloc[ind][text_cols[0]] + ' ' + \
                     inc_text_df.iloc[ind][text_cols[1]] + ' ' + \
                     inc_text_df.iloc[ind][text_cols[2]] + ' ' + \
                     inc_text_df.iloc[ind][text_cols[3]] + ' ' + \
                     inc_text_df.iloc[ind][text_cols[4]] + ' ' + \
                     inc_text_df.iloc[ind][text_cols[5]])
    text_data = inc_text_entry
    text_data = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - Guest \(Additional comments\)', '', text_data)
    text_data = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - System \(Additional comments\)', '', text_data)
    text_data = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \w+ \w+ \(Additional comments\)', '', text_data)
    text_data = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \w+ \w+ \(Staff work notes \(NERSC private\)\)', '', text_data)
    text_data = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - Guest \(Staff work notes \(NERSC private\)\)', '', text_data)
    text_data = re.sub(r'@\w+ \w+', '', text_data)
    text_data = re.sub(r'\d{4}-\d{2}-\d{2}', '',text_data)
    #text_data = re.sub(r'\d{2}:\d{2}:\d{2}', '', text_data) #remove system times
    #text_data = re.sub(r'\(Additional comments\)', '', text_data)
    #text_data = re.sub(r'\(Staff work notes \(NERSC private\)\)', '', text_data)
    text_data = text_data.lower()
    # remove html links
    text_data = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', '', text_data)
    #text_data = re.sub(r'\.','', text_data)
    text_data = re.sub(r',','', text_data)
    text_data = re.sub(r':','', text_data)
    text_data = re.sub(r'\?','', text_data)
    # removing quotes can be problematic
    #text_data = re.sub(r'\"','', text_data)
    #text_data = re.sub(r'\'','', text_data)
    text_data = text_data.split()
    wl = WordNetLemmatizer()
    text_data = [wl.lemmatize(word) for word in text_data if not word in set(stopwords.words('english'))]
    text_data = ' '.join(text_data)
    return text_data



In [10]:
#clean data set --> save as parquet 
CSV_OUTPUT_PATH = "/pscratch/sd/m/masare/sacct/perlmutter"
STEPS_CSV_PATH = "/pscratch/sd/m/masare/sacct/perlmutter/steps.csv"
JOBS_CSV_PATH = "/pscratch/sd/m/masare/sacct/perlmutter/jobs.csv"

STEPS_PARQUET_PATH = STEPS_CSV_PATH.replace(".csv", ".parquet")
JOBS_PARQUET_PATH = JOBS_CSV_PATH.replace(".csv", ".parquet")

START_DATE = "2023-01-01"
END_DATE = "2023-07-31"
SACCT_DELIMITER = "|||"
CSV_DELIMITER = "\t"
SACCT_FIELDS = [
    'Number',
    'Created',
    'Title',
    'Additional comments',
    'Close notes',
    'Comments and Work notes',
    'Staff work notes (NERSC private)'
]

In [11]:
def time_windows(start, end, width="Created"):
    t0 = pd.Timestamp(start)
    t1 = pd.Timestamp(end)
    dt = pd.Timedelta(width)
    starts = pd.date_range(t0, t1-dt, freq=width)
    ends = starts + pd.Timedelta(width)
    for s, e in zip(starts, ends):
        yield s, e

def sacct2csv(start, end, sacct_fields, csv_output_path, delimiter='|||', state=None, force=False):
    """ Run `sacct` and save the output to csv
    TODO: add --duplicates???
    """
    SLURM_TIME_FORMAT="%s"
    cmd = [
        "sacct",
        "--allusers",
        "--parsable",
        "--delimiter", delimiter,
        "--format", ",".join(sacct_fields),
        "--starttime", start,
        "--endtime", end
    ]
    if state:
        cmd += ["--state", f"{state}"]
    
    fout = f"{csv_output_path}/{start}.raw.csv"
    if pathlib.Path(fout).exists() and not force:
        return fout
        
    pathlib.Path(fout).parent.mkdir(parents=True, exist_ok=True)
    with open(fout, "w") as f:
        subprocess.run(cmd, env={"SLURM_TIME_FORMAT":"%s"}, stdout=f)
    return fout

In [12]:
days = list(time_windows(START_DATE, END_DATE))
for start, end in tqdm.tqdm(days):
    f = sacct2csv(start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"),
                  SACCT_FIELDS, CSV_OUTPUT_PATH, SACCT_DELIMITER)

ValueError: unit abbreviation w/o a number

In [ ]:
def flat_rows(p_in, delimiter):
    """Loop through rows
    compress the extra whitespace
    yield rows"""
    re_whitespace = re.compile(br"\s+")
    row_delimiter = (delimiter + '\n').encode()

    with open(p_in, "rb") as f_in:
        csv = f_in.read()
        start, end = 0, 0
        
        # skip the 1st row
        end = csv.find(row_delimiter, start)
        start = end + len(row_delimiter)
        
        while end != -1:
            end = csv.find(row_delimiter, start)
            if end != -1:
                row = re_whitespace.sub(b' ', csv[start:end]).strip()
                yield(row + b'\n')
            start = end + len(row_delimiter)


def decode_utf8(row):
    """Need replace here as users occassionally use weird chars
    e.g. `sacct -j 5697497`"""
    return row.decode("utf-8", "replace")


def decode_step_to_int(step):
    """Use the 32 bit int representation (lifted from Slurm source)
    for the 'special' steps"""
    
    if step == 'extern':
        return 0xFFFFFFFC
    elif step == 'batch':
        return 0xFFFFFFFB
    elif step == 'interactive':
        return 0xFFFFFFFA
    else:
        return int(step)
    
def decode_step_to_str(step):
    if step == 'extern':
        return '4294967292' # 0xFFFFFFFC
    elif step == 'batch':
        return '4294967291' # 0xFFFFFFFB
    elif step == 'interactive':
        return '4294967290' # 0xFFFFFFFA
    else:
        if step.isnumeric():
            return step
        else:
            raise ValueError(f"{step} not numeric")

def unpack_jobidraw(jobidraw):
    jobstep = -1
    hetjoboffset = 0
    
    fields = jobidraw.replace("+", ".").split(".")
    n = len(fields)

    hetjoboffset = "0"
    if n == 1:
        jobid = fields[0]
        jobstep = -1
    elif n == 2:
        jobid = fields[0]
        jobstep = decode_step_to_str(fields[1])
    elif n == 3:
        jobid = fields[0]
        jobstep = decode_step_to_str(fields[1])
        hetjoboffset = fields[2]
    
    return jobid, jobstep, hetjoboffset


def transform_step_row(cols, delimiter):
    """
    Could be <jobid> for an allocation
    Generally <jobid>.<stepid>+<HetJobOffset> for steps
        e.g. 13943741.2+1 
    """
    
    jobid, jobstep, hetjoboffset = unpack_jobidraw(cols[0])
    row = delimiter.join(
        [jobid, jobstep, hetjoboffset] + cols[1:])
    return row


def split_steps_and_jobs(row, delimiter, f_steps, f_jobs, d_out='\t'):
    cols = row.split(delimiter)
    jobidraw = cols[0]
    if '.' in jobidraw:
        f_steps.write(transform_step_row(cols, d_out))
    else:
        f_jobs.write(d_out.join(cols))


def write_header(f_in, delimiter, f_steps, f_jobs, d_out='\t'):
    header = f_in.readline().rstrip(delimiter + "\n").replace(delimiter, d_out)
    header = header.replace('Number', 'JobID')
    f_jobs.write(header + "\n")
    header_steps = d_out.join(
        ["JobID", "JobStep", "HetJobOffset"] + header.split(d_out)[1:]
    )
    f_steps.write(header_steps + "\n")

    
def check_field_count(row, delimiter, n):
    if len(row.split(delimiter)) != n:
        print("WARN:", row)

In [ ]:
raw_csvs = list(pathlib.Path(CSV_OUTPUT_PATH).glob("*.raw.csv"))

with (open(STEPS_CSV_PATH, "w") as f_steps,
      open(JOBS_CSV_PATH, "w") as f_jobs):
    
    with open(raw_csvs[0], "r") as csv0:
        write_header(csv0, SACCT_DELIMITER, f_steps, f_jobs, CSV_DELIMITER)
    
    for p_in in tqdm.tqdm(raw_csvs):
        for row in (decode_utf8(r) for r in flat_rows(p_in, SACCT_DELIMITER)):
            split_steps_and_jobs(row, SACCT_DELIMITER, f_steps, f_jobs, CSV_DELIMITER)


In [ ]:
dtypes = collections.defaultdict(lambda: str)
dtypes["JobID"] = np.int32
dtypes["Created"] = np.int32
dtypes["Title"] = str #np.int32
dtypes["Additional comments"] = str #np.int64
dtypes["Close notes"] = str #np.int16
dtypes["Comments and Work notes"] = str #np.int32
dtypes["Staff work notes (NERSC private)"] = str  #np.int64
#dtypes["Submit"] = np.int64
dtypes["Start"] = str # np.int64, str b/c it may not have started
dtypes["End"] = str # np.int64, str b/c it may not have started
#dtypes["ElapsedEaw"] = np.int64
#dtypes['HetJobOffset'] = np.int8

# Some columns are just always empty for steps so no need to keep those
# They will need to be joined back in from the jobs table
step_cols = [
    'JobID',
    'Created',
    'Title',
    'Additional comments',
    'Close notes',
    'Comments and Work notes',
    'Staff work notes (NERSC private)'
]


In [ ]:
df = pd.read_csv(
    STEPS_CSV_PATH,
    delimiter = CSV_DELIMITER,
    on_bad_lines = 'warn',
    dtype = dtypes,
    usecols = step_cols, 
    memory_map = True)
df.to_parquet(STEPS_PARQUET_PATH)

In [ ]:
del df